In [7]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import copy
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

filelist = []
for dirname, _, filenames in os.walk('C:/Users/zqzhang.2020/Desktop/algoTerm/code/data'):
    for filename in filenames:
        if filename.endswith(".in"):
            filelist.append(os.path.join(dirname, filename))

for filename in filelist:
    
    print('Process file: ' + filename)

    # Part 0: Reading Input File
    with open(filename, 'r') as data:
        first_line = data.readline()
        rows, cols, num_drones, num_turns, payload = first_line.split()
        rows, cols, num_drones, num_turns, payload = int(rows), int(cols), int(num_drones), int(num_turns), int(payload)
    
        second_line = data.readline()
        num_prodtypes = int(second_line)
        
        third_line = data.readline()
        prodtype_weights = [int(x) for x in third_line.split()]
        
        fourth_line = data.readline()
        num_warehouses = int(fourth_line)
        
        # 2D Array, row = warehouse, col = (x, y) coordinates
        warehouse_locs = []
        # 2D Array, row = warehouse, col = quantity of each prod
        warehouse_prods = []
        for i in range(num_warehouses):
            wh_coords = data.readline()
            warehouse_locs.append([int(x) for x in wh_coords.split()])
            prod_qty = data.readline()
            warehouse_prods.append([int(x) for x in prod_qty.split()])
        
        num_orders = data.readline()
        num_orders = int(num_orders)
        
        # 2D Array, row = order, col = (x, y) coordinates
        order_locs = []
        # 2D Array, row = warehouse, col = quantity of each prod type ordered
        order_prods = []
        for i in range(num_orders):
            order_coords = data.readline()
            order_locs.append([int(x) for x in order_coords.split()])
            num_prods_ordered = data.readline()
            num_prods_ordered = int(num_prods_ordered)
            
            prods_ordered = [0] * num_prodtypes
                        
            prodtype_ordered = data.readline()
            prodtype_ordered = [int(x) for x in prodtype_ordered.split()]
            
            for prodtype in prodtype_ordered:
                prods_ordered[prodtype] += 1
            
            order_prods.append(prods_ordered)
        
        prodtype_weights = np.array(prodtype_weights)
        warehouse_locs = np.array(warehouse_locs)
        warehouse_prods = np.array(warehouse_prods)
        order_locs = np.array(order_locs)
        order_prods = np.array(order_prods)
        
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session




Process file: C:/Users/zqzhang.2020/Desktop/algoTerm/code/data\busy_day.in


In [8]:
# calculate distance matrix
# coordinates of customers
coordinates = []
for i in range(len(warehouse_locs)):
    coordinates.append(warehouse_locs[i].tolist())
for i in range(len(order_locs)):
    coordinates.append(order_locs[i].tolist())
coordinates=np.array(coordinates)

# computing distance bewteen customers, namely c_ij, 0 is depot
def getdistmat_cij(coordinates):
    num = coordinates.shape[0]
    distmat = np.zeros((num,num))
    
    # computing the distance using L_2norm
    for i in range(num):
        for j in range(i, num):
            distmat[i][j] = distmat[j][i] = \
                      round(np.linalg.norm(coordinates[i] - coordinates[j]),4)
    return distmat
c_ij = getdistmat_cij(coordinates)

In [9]:
# functions

# drone_route -> merged_route
# for the former, it is actually the command for drones
# for the latter, the location index for orders +10 to fit the distance matrix

def dr2mr(r):
    r = r[:]
    r.append(['x','x','x','x','x'])
    sol = []
    loc = -1
    temp_list = []
    prod_list = []
    for i in range(len(r)):
        if r[i][2] != loc or r[i][1] != r[i-1][1]:
            if loc != -1:
                sol.append(copy.deepcopy(temp_list))
                temp_list.clear()
            prod_list.clear()
            prod_list.append((r[i][3],r[i][4]))
            temp_list.append(r[i][0])
            temp_list.append(r[i][1])
            if r[i][1] == 'D':
                temp_list.append(r[i][2]+10)     # for orders +10
            else:
                temp_list.append(r[i][2])    
            temp_list.append(prod_list)
           
            loc = r[i][2]
        else:
            prod_list.append((r[i][3],r[i][4]))
        
    return sol



# merged_route -> drone_route
def mr2dr(r):
    sol = []
    for i in range(len(r)):
        for j in range(len(r[i][3])):
            if r[i][1] == 'L': 
                sol.append([r[i][0], r[i][1],r[i][2],r[i][3][j][0],r[i][3][j][1]])
            else:
                sol.append([r[i][0], r[i][1],r[i][2]-10,r[i][3][j][0],r[i][3][j][1]])
    return sol

# calculate route total distance
def getroute_length(r):
    l = 0
    
    for i in range(len(r) - 1):
        l += c_ij[r[i][2]][r[i+1][2]]
    return l


# 1-opt move and local search
# check whether index i can be inserted after index j
def is_insert_feasible(r,i,j):
    dic={}
    if r[i][1]=='D':
        for m in range(len(r[i][3])):
            dic[r[i][3][m][0]]=r[i][3][m][1]*(-1)
        for m in range(j+1):
            for n in range(len(r[m][3])):
                if  r[m][3][n][0] in dic.keys():
                    if r[m][1]=='L':
                        dic[r[m][3][n][0]]+=r[m][3][n][1]
                    else:
                        dic[r[m][3][n][0]]-=r[m][3][n][1]
        if min(dic.values())<0:
            return False
    else:
        for m in range(len(r[i][3])):
            dic[r[i][3][m][0]]=0
        for m in range(j+1):
            if m==i:
                continue
            for n in range(len(r[m][3])):
                if  r[m][3][n][0] in dic.keys():
                    if r[m][1]=='L':
                        dic[r[m][3][n][0]]+=r[m][3][n][1]
                    else:
                        dic[r[m][3][n][0]]-=r[m][3][n][1]
        if min(dic.values())<0:
            return False
    return True       

# calculate the delta distance
def delta_dis(r, i, j):
    if i == 0:
        if j != len(r) - 1:
            return - c_ij[r[i][2]][r[i+1][2]] - c_ij[r[j][2]][r[j+1][2]] + c_ij[r[j][2]][r[i][2]] + c_ij[r[i][2]][r[j+1][2]]
        else:
            return - c_ij[r[i][2]][r[i+1][2]] + c_ij[r[i][2]][r[j][2]]
    elif i != len(r)-1:
        if j != len(r) - 1:
            return - c_ij[r[i-1][2]][r[i][2]] - c_ij[r[i][2]][r[i+1][2]] - c_ij[r[j][2]][r[j+1][2]] + c_ij[r[i-1][2]][r[i+1][2]] + c_ij[r[i][2]][r[j][2]] + c_ij[r[i][2]][r[j+1][2]]
        else:
            return - c_ij[r[i-1][2]][r[i][2]] - c_ij[r[i][2]][r[i+1][2]] + c_ij[r[i][2]][r[j][2]]
    else:
        return - c_ij[r[i-1][2]][r[i][2]] - c_ij[r[j][2]][r[j+1][2]] + c_ij[r[i][2]][r[j][2]] + c_ij[r[j+1][2]][r[i][2]]


# local search by 1-opt with only downhill (better solution)
# time_0_flag = 0 means at time turn 0, do not move location 0

def local_search_1opt(r, time_0_flag = 0):
    for i in range(len(r)):
        for j in range(len(r)):
            if i == j or (time_0_flag == 0 and i == 0):
                continue
            else:
                if is_insert_feasible(r, i, j):
                    if delta_dis(r,i,j) < 0:
                        temp = r[i]
                        del r[i]
                        if i < j:
                            r.insert(j,temp)
                        else:
                            r.insert(j+1,temp)
    return None
  



def local_search_1opt(r, time_0_flag = 0):
    for i in range(len(r)):
        for j in range(len(r)):
            if i == j or (time_0_flag == 0 and i == 0):
                continue
            else:
                if is_insert_feasible(r, i, j):
                    if delta_dis(r,i,j) < 0:
                        temp = r[i]
                        del r[i]
                        if i < j:
                            r.insert(j,temp)
                        else:
                            r.insert(j+1,temp)
    return None

def r2r(r, time_0_flag = 0):
    BKS = copy.deepcopy(r)
    BKS_value = getroute_length(BKS)
    
    records = BKS_value * 0.01
    
    
    for i in range(len(r)):
        for j in range(len(r)):
            if i == j or (time_0_flag == 0 and i == 0):
                continue
            else:
                if is_insert_feasible(r, i, j):
                    if delta_dis(r,i,j) < 0:
                        temp = r[i]
                        del r[i]
                        if i < j:
                            r.insert(j,temp)
                        else:
                            r.insert(j+1,temp)
                        BKS = r
                        BKS_value = getroute_length(BKS)
                        records = BKS_value * 0.05
                    elif delta_dis(r,i,j) < records:
                        temp = r[i]
                        del r[i]
                        if i < j:
                            r.insert(j,temp)
                        else:
                            r.insert(j+1,temp)
    r = BKS
    return None
    
def local_search(r, time_0_flag = 0):
    BKS_Value = getroute_length(r)
    while 1:
        r2r(r)
        local_search_1opt(r)
        current = getroute_length(r)
        delta = BKS_Value - current
        if delta > 0:
            BKS_Value = current
        
        if delta <= 1e-5:
            break
    return None


###################################################  
def recalcDroneLocTime(partialSoln, orig_avail_time, orig_drone_loc, warehouse_locs, order_locs):
    
    avail_time = orig_avail_time
    drone_loc = orig_drone_loc
    
    for command in partialSoln:
        drone_id = command[0]
        if (command[1] == 'L') or (command[1] == 'U'):
            wh_id = command[2]
            curr_wh_loc = warehouse_locs[wh_id]
            dist_wh = np.linalg.norm(curr_wh_loc - drone_loc, 2)
            avail_time += (np.ceil(dist_wh) + 1)
            drone_loc = warehouse_locs[wh_id]
        elif command[1] == 'D':
            order_id = command[2]
            curr_order_loc = order_locs[order_id]
            dist_order = np.linalg.norm(curr_order_loc - drone_loc, 2)
            avail_time += (np.ceil(dist_order) + 1)
            drone_loc = order_locs[order_id]
    
    return avail_time, drone_loc

def partialSolnStrOutput(list_commands):    
    convertedPartialSoln = []
    for command in list_commands:
        if command[1] == 'W':
            convertedPartialSoln.append(str(command[0]) + " W " + str(command[2]))
        else:
            convertedPartialSoln.append(str(command[0]) + " " + str(command[1]) + " " + str(command[2]) + " " + str(command[3]) + " " + str(command[4]))
    return convertedPartialSoln
            

In [10]:
solution = [] # List of string

# Drone's timing
drone_avail_time = np.array([0] * num_drones)
drone_curr_loc = np.tile(warehouse_locs[0], (num_drones, 1))

skip_counter = 0
# Any drone still available before total number of turns
while (np.any(drone_avail_time < num_turns) and (np.sum(order_prods) > 0) and (skip_counter < len(drone_avail_time))):
    
    # Choose the earliest available drone, skipping those without solutions
    drone_no = np.argsort(drone_avail_time)[skip_counter]
    # No drone to plan anymore
    if drone_avail_time[drone_no] >= num_turns:
        break
    
    print("Planning for drone " + str(drone_no) + " available at time " + str(drone_avail_time[drone_no]))
    
    # Item Planning Phase    
    avail_time = drone_avail_time[drone_no]
    # Record available space in the drone
    avail_space = payload
    # Current drone solution
    drone_solution = []
    
    order_prods_plan = np.copy(order_prods)
    warehouse_prods_plan = np.copy(warehouse_prods)

    # Aggregate weights per order
    order_weights = order_prods_plan * prodtype_weights # 2D matrix, row = order, col = prodtype
    weights_per_order = np.sum(order_weights, axis = 1) # 1D vector, length = number of orders
        
    load_items = dict() # key = prodtype, val = units
    deliver_items = dict() # key = orderNo, val = load_dict
    
    # Phase 1: Order and Product Assignment
    for orderNo in weights_per_order.argsort():
        
        # Filter to the ordered product types
        ordered_prodtypes = np.where(order_prods_plan[orderNo] > 0)[0]
        ordered_prod_unitweight = prodtype_weights[ordered_prodtypes]
                
        # Heaviest item first
        for idx in ordered_prod_unitweight.argsort()[::-1]:
            
            # If available space is less than the unit weight of this product
            if avail_space < ordered_prod_unitweight[idx]:
                break
            
            units_to_load = avail_space // ordered_prod_unitweight[idx]
            
            if units_to_load > order_prods_plan[orderNo, ordered_prodtypes[idx]]:
                units_to_load = order_prods_plan[orderNo, ordered_prodtypes[idx]]
                order_prods_plan[orderNo, ordered_prodtypes[idx]] = 0
            else:
                order_prods_plan[orderNo, ordered_prodtypes[idx]] -= units_to_load
            
            avail_space -= (units_to_load * ordered_prod_unitweight[idx])
            
            if units_to_load > 0:
                if orderNo in deliver_items:
                    if ordered_prodtypes[idx] in deliver_items[orderNo]:
                        deliver_items[orderNo][ordered_prodtypes[idx]] += units_to_load
                    else:
                        deliver_items[orderNo][ordered_prodtypes[idx]] = units_to_load
                else:
                    deliver_items[orderNo] = {ordered_prodtypes[idx]: units_to_load}
                
                if ordered_prodtypes[idx] in load_items:
                    load_items[ordered_prodtypes[idx]] += units_to_load
                else:
                    load_items[ordered_prodtypes[idx]] = units_to_load
    
    # Warehouse route planning
    drone_loc_plan = drone_curr_loc[drone_no]
    
    while ((len(load_items) > 0) and (avail_time < num_turns)):
        # Distance from drone to warehouse
        dist_warehouse = np.linalg.norm(warehouse_locs - drone_loc_plan, 2, axis = 1)
        
        for warehouse_id in dist_warehouse.argsort():
            # Available product types in this warehouse
            warehouse_avail_prods = np.where(warehouse_prods_plan[warehouse_id] > 0)[0]
            # Remaining product types to load
            target_items = np.array(list(load_items.keys()))
            
            warehouse_load_prodtypes = target_items[np.in1d(target_items, warehouse_avail_prods)]
            
            if len(warehouse_load_prodtypes) > 0:
                
                for prodtype in warehouse_load_prodtypes:
                    
                    # Load minimum of the required number and available number in the warehouse
                    load_num_units = min(load_items[prodtype], warehouse_prods_plan[warehouse_id, prodtype])
                    
                    load_items[prodtype] -= load_num_units
                    if load_items[prodtype] == 0:
                        del load_items[prodtype]
                    
                    warehouse_prods_plan[warehouse_id, prodtype] -= load_num_units
                    
                    avail_time += (np.ceil(dist_warehouse[warehouse_id]) + 1)
                    drone_loc_plan = warehouse_locs[warehouse_id]
                    
                    #drone_solution.append(str(drone_no) + " L " + str(warehouse_id) + " " + str(prodtype) + " " + str(load_num_units))
                    drone_solution.append([drone_no, "L", warehouse_id, prodtype, load_num_units])
                
                break # Break, have to recalculate distance
    
    # Phase 2: Warehouse & Customer delivery route planning
    while ((len(deliver_items) > 0) and (avail_time < num_turns)):
        
        cust_order_no = np.array(list(deliver_items.keys()))
        cust_order_locs = order_locs[cust_order_no]
        
        # Distance from drone to customers
        dist_cust = np.linalg.norm(cust_order_locs - drone_loc_plan, 2, axis = 1)
        
        # Go to nearest order for delivery
        delivery_cust_idx = dist_cust.argmin()
        delivery_order_no = cust_order_no[delivery_cust_idx]
        
        for prodtype, units in deliver_items[delivery_order_no].items():
            
            avail_time += (np.ceil(dist_cust[delivery_cust_idx]) + 1)
            drone_loc_plan = cust_order_locs[delivery_cust_idx]
            
            #drone_solution.append(str(drone_no) + " D " + str(delivery_order_no) + " " + str(prodtype) + " " + str(units))
            drone_solution.append([drone_no, "D", delivery_order_no, prodtype, units])
        
        del deliver_items[delivery_order_no]
    
    # local serach for route planning 
    
    
    
    # Do heuristic search here
    mr = dr2mr(drone_solution)
    local_search(mr)
    
    drone_solution = mr2dr(mr)
    
    # Update avail_time and drone_loc_plan based on the updated partial solution
    avail_time, drone_loc_plan = recalcDroneLocTime(drone_solution, drone_avail_time[drone_no], 
                                                    drone_curr_loc[drone_no], warehouse_locs, 
                                                    order_locs)
    
    # Convert partial solution to list of strings
    drone_solution = partialSolnStrOutput(drone_solution)
    
    
    
    if avail_time <= num_turns:
        #print("soln found")
        solution += drone_solution
        drone_curr_loc[drone_no] = drone_loc_plan
        drone_avail_time[drone_no] = avail_time        
        # Persist np arrays
        order_prods = order_prods_plan
        warehouse_prods = warehouse_prods_plan
        # Counter to keep track how many rounds of no solution found
        skip_counter = 0
    else:
        skip_counter += 1
    
if (np.sum(order_prods) > 0):
    wait_drones = np.where(drone_avail_time < num_turns)[0]
    for wait_drone in wait_drones:
        solution.append(str(wait_drone) + " W " + str(num_turns - drone_avail_time[wait_drone]))

    drone_avail_time[wait_drones] = num_turns

Planning for drone 0 available at time 0
Planning for drone 14 available at time 0
Planning for drone 29 available at time 0
Planning for drone 13 available at time 0
Planning for drone 27 available at time 0
Planning for drone 26 available at time 0
Planning for drone 12 available at time 0
Planning for drone 25 available at time 0
Planning for drone 11 available at time 0
Planning for drone 24 available at time 0
Planning for drone 23 available at time 0
Planning for drone 10 available at time 0
Planning for drone 21 available at time 0
Planning for drone 22 available at time 0
Planning for drone 9 available at time 0
Planning for drone 19 available at time 0
Planning for drone 8 available at time 0
Planning for drone 17 available at time 0
Planning for drone 16 available at time 0
Planning for drone 1 available at time 0
Planning for drone 2 available at time 0
Planning for drone 15 available at time 0
Planning for drone 28 available at time 0
Planning for drone 18 available at time

Planning for drone 29 available at time 3450
Planning for drone 14 available at time 3457
Planning for drone 19 available at time 3463
Planning for drone 26 available at time 3480
Planning for drone 4 available at time 3502
Planning for drone 3 available at time 3541
Planning for drone 12 available at time 3541
Planning for drone 16 available at time 3547
Planning for drone 6 available at time 3567
Planning for drone 23 available at time 3569
Planning for drone 22 available at time 3575
Planning for drone 10 available at time 3611
Planning for drone 5 available at time 3626
Planning for drone 24 available at time 3699
Planning for drone 28 available at time 3701
Planning for drone 20 available at time 3712
Planning for drone 15 available at time 3721
Planning for drone 0 available at time 3732
Planning for drone 8 available at time 3748
Planning for drone 27 available at time 3801
Planning for drone 1 available at time 3819
Planning for drone 14 available at time 3838
Planning for dron

Planning for drone 18 available at time 6749
Planning for drone 15 available at time 6848
Planning for drone 4 available at time 6867
Planning for drone 21 available at time 6869
Planning for drone 5 available at time 6891
Planning for drone 6 available at time 6905
Planning for drone 19 available at time 6909
Planning for drone 29 available at time 6932
Planning for drone 23 available at time 6940
Planning for drone 1 available at time 6959
Planning for drone 14 available at time 6973
Planning for drone 2 available at time 6985
Planning for drone 0 available at time 6997
Planning for drone 16 available at time 7003
Planning for drone 25 available at time 7014
Planning for drone 26 available at time 7027
Planning for drone 24 available at time 7064
Planning for drone 7 available at time 7108
Planning for drone 28 available at time 7108
Planning for drone 4 available at time 7134
Planning for drone 3 available at time 7169
Planning for drone 11 available at time 7179
Planning for drone 

Planning for drone 28 available at time 10291
Planning for drone 9 available at time 10311
Planning for drone 23 available at time 10323
Planning for drone 17 available at time 10342
Planning for drone 20 available at time 10371
Planning for drone 29 available at time 10372
Planning for drone 5 available at time 10388
Planning for drone 18 available at time 10390
Planning for drone 21 available at time 10400
Planning for drone 13 available at time 10402
Planning for drone 7 available at time 10436
Planning for drone 8 available at time 10475
Planning for drone 2 available at time 10478
Planning for drone 4 available at time 10499
Planning for drone 22 available at time 10506
Planning for drone 16 available at time 10510
Planning for drone 4 available at time 10552
Planning for drone 24 available at time 10560
Planning for drone 29 available at time 10593
Planning for drone 19 available at time 10599
Planning for drone 3 available at time 10604
Planning for drone 17 available at time 10

Planning for drone 7 available at time 13681
Planning for drone 6 available at time 13685
Planning for drone 14 available at time 13689
Planning for drone 2 available at time 13702
Planning for drone 20 available at time 13712
Planning for drone 10 available at time 13742
Planning for drone 27 available at time 13783
Planning for drone 23 available at time 13838
Planning for drone 15 available at time 13855
Planning for drone 26 available at time 13896
Planning for drone 25 available at time 13906
Planning for drone 17 available at time 13940
Planning for drone 13 available at time 13945
Planning for drone 24 available at time 13948
Planning for drone 20 available at time 13951
Planning for drone 22 available at time 13963
Planning for drone 9 available at time 13990
Planning for drone 29 available at time 13992
Planning for drone 21 available at time 13999
Planning for drone 7 available at time 14011
Planning for drone 3 available at time 14078
Planning for drone 28 available at time 

...

Planning for drone 5 available at time 48916
Planning for drone 2 available at time 48920
Planning for drone 1 available at time 48957
Planning for drone 28 available at time 49001
Planning for drone 24 available at time 49016
Planning for drone 8 available at time 49049
Planning for drone 1 available at time 49096
Planning for drone 6 available at time 49096
Planning for drone 9 available at time 49096
Planning for drone 25 available at time 49105
Planning for drone 20 available at time 49135
Planning for drone 21 available at time 49190
Planning for drone 22 available at time 49191
Planning for drone 24 available at time 49207
Planning for drone 0 available at time 49219
Planning for drone 2 available at time 49227
Planning for drone 26 available at time 49238
Planning for drone 9 available at time 49260
Planning for drone 19 available at time 49285
Planning for drone 17 available at time 49289
Planning for drone 13 available at time 49290
Planning for drone 18 available at time 4934

Planning for drone 21 available at time 52102
Planning for drone 13 available at time 52103
Planning for drone 9 available at time 52148
Planning for drone 28 available at time 52186
Planning for drone 8 available at time 52190
Planning for drone 15 available at time 52224
Planning for drone 0 available at time 52232
Planning for drone 19 available at time 52240
Planning for drone 14 available at time 52258
Planning for drone 12 available at time 52258
Planning for drone 1 available at time 52263
Planning for drone 27 available at time 52274
Planning for drone 23 available at time 52283
Planning for drone 10 available at time 52296
Planning for drone 17 available at time 52338
Planning for drone 29 available at time 52344
Planning for drone 0 available at time 52381
Planning for drone 22 available at time 52394
Planning for drone 25 available at time 52394
Planning for drone 4 available at time 52418
Planning for drone 14 available at time 52423
Planning for drone 11 available at time 

Planning for drone 16 available at time 55225
Planning for drone 4 available at time 55229
Planning for drone 14 available at time 55309
Planning for drone 28 available at time 55318
Planning for drone 23 available at time 55330
Planning for drone 13 available at time 55332
Planning for drone 19 available at time 55338
Planning for drone 5 available at time 55344
Planning for drone 27 available at time 55346
Planning for drone 20 available at time 55361
Planning for drone 17 available at time 55379
Planning for drone 26 available at time 55382
Planning for drone 21 available at time 55384
Planning for drone 18 available at time 55417
Planning for drone 29 available at time 55423
Planning for drone 1 available at time 55429
Planning for drone 24 available at time 55447
Planning for drone 22 available at time 55450
Planning for drone 25 available at time 55451
Planning for drone 12 available at time 55455
Planning for drone 6 available at time 55459
Planning for drone 3 available at time

Planning for drone 15 available at time 58412
Planning for drone 29 available at time 58416
Planning for drone 8 available at time 58426
Planning for drone 3 available at time 58429
Planning for drone 23 available at time 58449
Planning for drone 11 available at time 58474
Planning for drone 4 available at time 58499
Planning for drone 25 available at time 58519
Planning for drone 7 available at time 58526
Planning for drone 26 available at time 58539
Planning for drone 5 available at time 58551
Planning for drone 21 available at time 58554
Planning for drone 16 available at time 58558
Planning for drone 12 available at time 58564
Planning for drone 22 available at time 58639
Planning for drone 1 available at time 58641
Planning for drone 0 available at time 58697
Planning for drone 2 available at time 58702
Planning for drone 17 available at time 58709
Planning for drone 7 available at time 58741
Planning for drone 3 available at time 58746
Planning for drone 27 available at time 5874

Planning for drone 14 available at time 61634
Planning for drone 0 available at time 61641
Planning for drone 12 available at time 61669
Planning for drone 24 available at time 61719
Planning for drone 1 available at time 61722
Planning for drone 16 available at time 61724
Planning for drone 25 available at time 61747
Planning for drone 21 available at time 61766
Planning for drone 23 available at time 61769
Planning for drone 7 available at time 61771
Planning for drone 1 available at time 61811
Planning for drone 19 available at time 61852
Planning for drone 22 available at time 61860
Planning for drone 6 available at time 61868
Planning for drone 2 available at time 61878
Planning for drone 26 available at time 61888
Planning for drone 11 available at time 61891
Planning for drone 27 available at time 61901
Planning for drone 23 available at time 61926
Planning for drone 24 available at time 61928
Planning for drone 3 available at time 61939
Planning for drone 20 available at time 6

In [5]:
with open('C:/Users/zqzhang.2020/Desktop/algoTerm/code/output.txt', 'w') as fd:
    fd.write(str(len(solution)) + '\n')
    for writeLine in solution:
        fd.write(writeLine + '\n')